# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from scipy import stats
import math

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks = stocks[~stocks['Ticker'].isin(['DISCA', 'HFC','VIAC','WLTW'])]
stocks.reset_index(inplace = True, drop = True)
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
496,YUM
497,ZBH
498,ZBRA
499,ZION


In [3]:
from secrets_algo import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [4]:
symbol = 'aapl'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'avgTotalVolume': 66517260, 'calculationPrice': 'close', 'change': -2.58, 'changePercent': -0.01543, 'close': 164.25, 'closeSource': 'oifalifc', 'closeTime': 1681608074539, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': 162.42, 'delayedPriceTime': 1708408444724, 'extendedChange': 0.06, 'extendedChangePercent': 0.0004, 'extendedPrice': 162.24, 'extendedPriceTime': 1675280660015, 'high': 168.78, 'highSource': 'tmnd51laryd iep  iuceee', 'highTime': 1697323868671, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexClose': 165.42, 'iexCloseTime': 1733307256521, 'iexLastUpdated': 1735692385563, 'iexMarketPercent': 0.01969475332075288, 'iexOpen': 167.08, 'iexOpenTime': 1682603218985, 'iexRealtimePrice': 163.79, 'iexRealtimeSize': 407, 'iexVolume': 1517386, 'lastTradeTime': 1690224592152, 'latestPrice': 163.78, 'latestSource': 'Close', 'latestTime': 'August 30, 2022', 'latestUpdate': 1716880979015, 'latestVolume': 78028631, 'low': 164.25, 'lowSource': 'p

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [5]:
price = data['latestPrice']
pe_ratio = data['peRatio']
pe_ratio

27.1

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [6]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [7]:
final_dataframe = pd.DataFrame(columns= my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    batch_data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
         final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    batch_data[symbol]['quote']['latestPrice'],
                    batch_data[symbol]['quote']['peRatio'],
                    'N/A'
                ], index= my_columns
            ), ignore_index= True
         )


final_dataframe

C:\Users\tyzp\AppData\Local\Temp/ipykernel_7596/2838065448.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\tyzp\AppData\Local\Temp/ipykernel_7596/2838065448.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\tyzp\AppData\Local\Temp/ipykernel_7596/2838065448.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\tyzp\AppData\Local\Temp/ipykernel_7596/2838065448.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\tyzp\AppData\Local\Temp

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,130.32,30.94,N/A
1,AAL,13.52,-4.7,N/A
2,AAP,173.22,20.28,N/A
3,AAPL,166.35,26.46,N/A
4,ABBV,141.91,19.45,N/A
...,...,...,...,...
496,YUM,112.47,22.9,N/A
497,ZBH,107.93,102.64,N/A
498,ZBRA,306.39,33.69,N/A
499,ZION,57.4,6.41,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [8]:
final_dataframe = final_dataframe.sort_values('Price-to-Earnings Ratio')
final_dataframe


,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
237,ILMN,213.11,-1773.17,N/A
446,TWTR,39.92,-275.56,N/A
291,LYV,91.5,-269.46,N/A
336,NOV,18.21,-78.49,N/A
207,GPS,9.87,-52.66,N/A
...,...,...,...,...
203,GOOG,111.96,None,N/A
252,J,129.56,None,N/A
346,NWS,18.13,None,N/A
440,TROW,124.18,None,N/A


In [9]:
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'].notna()]
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe.reset_index(inplace = True, drop = True)
final_dataframe = final_dataframe[0:50]
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,42.04,2.99,N/A
1,AIG,54.6,3.58,N/A
2,KSS,28.95,4.29,N/A
3,NUE,139.7,4.36,N/A
4,PVH,64.1,4.58,N/A
5,PHM,41.65,4.62,N/A
6,DHI,73.39,4.69,N/A
7,PFG,77.35,4.76,N/A
8,APA,40.46,4.78,N/A
9,MGM,34.47,4.8,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [10]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [11]:
portfolio_input()

You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [12]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for row in final_dataframe.index:
    final_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe.loc[row, 'Price'])

final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,42.04,2.99,11893
1,AIG,54.6,3.58,9157
2,KSS,28.95,4.29,17271
3,NUE,139.7,4.36,3579
4,PVH,64.1,4.58,7800
5,PHM,41.65,4.62,12004
6,DHI,73.39,4.69,6812
7,PFG,77.35,4.76,6464
8,APA,40.46,4.78,12357
9,MGM,34.47,4.8,14505


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [13]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()


In [14]:
#print(data['AAPL']['advanced-stats'])

#Price-to-earnings ratio
pe_ratio = data[symbol]['quote']['peRatio']

#Price-to-book ratio
##  /stock/{symbol}/advanced-stats/

pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#Price-to-sales ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

#Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
ev = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']

ev_to_ebitda = ev / ebitda

#Enterprise Value divided by Gross Profit (EV/GP)
gp = data[symbol]['advanced-stats']['grossProfit']

ev_to_gp = ev / gp
ev_to_gp

15.548323633050257

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [15]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earnings Ratio', 
    'PE Percentile', 
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
    ]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    



    for symbol in symbol_string.split(','):
       
        ev = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gp = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = ev/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN

        try:
            ev_to_gp = ev/gp
        except TypeError:
            ev_to_gp = np.NaN


        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'], 
                'N/A', 
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gp,
                'N/A',
                'N/A'
                
            ],
            index = rv_columns),
            ignore_index = True
        )

C:\Users\tyzp\AppData\Local\Temp/ipykernel_7596/3868124745.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
C:\Users\tyzp\AppData\Local\Temp/ipykernel_7596/3868124745.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
C:\Users\tyzp\AppData\Local\Temp/ipykernel_7596/3868124745.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
C:\Users\tyzp\AppData\Local\Temp/ipykernel_7596/3868124745.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
C:\Users\tyzp\AppData\Local\Temp/ipykernel_7596/3868

In [16]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,131.8,N/A,32.38,N/A,7.63,N/A,5.89,N/A,22.597971,N/A,11.344021,N/A,N/A
1,AAL,13.67,N/A,-4.8,N/A,-1.05,N/A,0.223,N/A,106.444325,N/A,0.82848,N/A,N/A
2,AAP,172.86,N/A,19.83,N/A,3.78,N/A,0.9527,N/A,10.958155,N/A,2.273113,N/A,N/A
3,AAPL,163.1,N/A,27.54,N/A,45.22,N/A,6.73,N/A,19.877669,N/A,15.981806,N/A,N/A
4,ABBV,137.56,N/A,19.39,N/A,17.05,N/A,4.35,N/A,11.265791,N/A,7.409769,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,116.3,N/A,22.75,N/A,-3.82,N/A,4.8,N/A,19.095356,N/A,8.852074,N/A,N/A
497,ZBH,108.48,N/A,101.19,N/A,1.92,N/A,3.08,N/A,16.792435,N/A,5.380625,N/A,N/A
498,ZBRA,303.15,N/A,32.83,N/A,6.6,N/A,2.77,N/A,16.880452,N/A,7.057445,N/A,N/A
499,ZION,57.9,N/A,6.39,N/A,1.69,N/A,2.4,N/A,4.996011,N/A,2.451047,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [17]:
#rv_dataframe[rv_dataframe.isnull()]
rv_dataframe[rv_dataframe.isnull().any(axis = 1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
28,ALXN,189.1,N/A,60.35,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,288.22,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,290.39,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
88,CERN,99.22,N/A,49,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11,N/A,10.03,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
135,DISCK,25.18,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
164,ETFC,50.27,N/A,14.69,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
185,FLIR,58.34,N/A,32.97,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
189,FOX,33.1,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
192,FRT,105.83,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [18]:
rv_dataframe.columns

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'Price-to-Earnings Ratio',
       'PE Percentile', 'Price-to-Book Ratio', 'PB Percentile',
       'Price-to-Sales Ratio', 'PS Percentile', 'EV/EBITDA',
       'EV/EBITDA Percentile', 'EV/GP', 'EV/GP Percentile', 'RV Score'],
      dtype='object')

In [19]:
ratios_columns = ['Price-to-Earnings Ratio',
       'Price-to-Book Ratio', 
       'Price-to-Sales Ratio', 'EV/EBITDA',
       'EV/GP']

In [20]:
for column in ratios_columns:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)


Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [21]:
rv_dataframe[rv_dataframe.isnull().any(axis = 1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [22]:
### Filter out negative PS, PB

rv_dataframe = rv_dataframe[rv_dataframe['Price-to-Earnings Ratio'] > 0]
rv_dataframe = rv_dataframe[rv_dataframe['Price-to-Book Ratio'] > 0]
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,131.8,N/A,32.38,N/A,7.63,N/A,5.8900,N/A,22.597971,N/A,11.344021,N/A,N/A
2,AAP,172.86,N/A,19.83,N/A,3.78,N/A,0.9527,N/A,10.958155,N/A,2.273113,N/A,N/A
3,AAPL,163.1,N/A,27.54,N/A,45.22,N/A,6.7300,N/A,19.877669,N/A,15.981806,N/A,N/A
4,ABBV,137.56,N/A,19.39,N/A,17.05,N/A,4.3500,N/A,11.265791,N/A,7.409769,N/A,N/A
5,ABC,148.77,N/A,17.40,N/A,138.13,N/A,0.1333,N/A,9.490256,N/A,3.970528,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,XYL,94.55,N/A,41.35,N/A,5.31,N/A,3.3000,N/A,21.900116,N/A,9.025940,N/A,N/A
497,ZBH,108.48,N/A,101.19,N/A,1.92,N/A,3.0800,N/A,16.792435,N/A,5.380625,N/A,N/A
498,ZBRA,303.15,N/A,32.83,N/A,6.60,N/A,2.7700,N/A,16.880452,N/A,7.057445,N/A,N/A
499,ZION,57.9,N/A,6.39,N/A,1.69,N/A,2.4000,N/A,4.996011,N/A,2.451047,N/A,N/A


In [23]:
metrics = {
    'Price-to-Earnings Ratio': 'PE Percentile', 
    'Price-to-Book Ratio': 'PB Percentile',
    'Price-to-Sales Ratio': 'PS Percentile',
    'EV/EBITDA': 'EV/EBITDA Percentile',
    'EV/GP': 'EV/GP Percentile'
}

for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,131.8,N/A,32.38,74.392936,7.63,75.275938,5.8900,82.560706,22.597971,84.768212,11.344021,81.456954,N/A
2,AAP,172.86,N/A,19.83,46.467991,3.78,51.214128,0.9527,14.128035,10.958155,34.878587,2.273113,14.128035,N/A
3,AAPL,163.1,N/A,27.54,66.225166,45.22,97.571744,6.7300,84.878587,19.877669,77.262693,15.981806,93.598234,N/A
4,ABBV,137.56,N/A,19.39,44.591611,17.05,92.494481,4.3500,73.951435,11.265791,36.644592,7.409769,55.187638,N/A
5,ABC,148.77,N/A,17.40,39.293598,138.13,99.116998,0.1333,0.220751,9.490256,28.03532,3.970528,28.03532,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,XYL,94.55,N/A,41.35,84.326711,5.31,64.238411,3.3000,58.057395,21.900116,81.898455,9.025940,71.302428,N/A
497,ZBH,108.48,N/A,101.19,97.792494,1.92,21.743929,3.0800,55.298013,16.792435,68.653422,5.380625,40.397351,N/A
498,ZBRA,303.15,N/A,32.83,74.613687,6.60,71.302428,2.7700,48.785872,16.880452,68.874172,7.057445,52.317881,N/A
499,ZION,57.9,N/A,6.39,5.298013,1.69,16.556291,2.4000,43.046358,4.996011,8.609272,2.451047,15.011038,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [24]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,131.8,N/A,32.38,74.392936,7.63,75.275938,5.8900,82.560706,22.597971,84.768212,11.344021,81.456954,79.690949
2,AAP,172.86,N/A,19.83,46.467991,3.78,51.214128,0.9527,14.128035,10.958155,34.878587,2.273113,14.128035,32.163355
3,AAPL,163.1,N/A,27.54,66.225166,45.22,97.571744,6.7300,84.878587,19.877669,77.262693,15.981806,93.598234,83.907285
4,ABBV,137.56,N/A,19.39,44.591611,17.05,92.494481,4.3500,73.951435,11.265791,36.644592,7.409769,55.187638,60.573951
5,ABC,148.77,N/A,17.40,39.293598,138.13,99.116998,0.1333,0.220751,9.490256,28.03532,3.970528,28.03532,38.940397
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,XYL,94.55,N/A,41.35,84.326711,5.31,64.238411,3.3000,58.057395,21.900116,81.898455,9.025940,71.302428,71.96468
497,ZBH,108.48,N/A,101.19,97.792494,1.92,21.743929,3.0800,55.298013,16.792435,68.653422,5.380625,40.397351,56.777042
498,ZBRA,303.15,N/A,32.83,74.613687,6.60,71.302428,2.7700,48.785872,16.880452,68.874172,7.057445,52.317881,63.178808
499,ZION,57.9,N/A,6.39,5.298013,1.69,16.556291,2.4000,43.046358,4.996011,8.609272,2.451047,15.011038,17.704194


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [25]:
rv_dataframe.sort_values('RV Score', ascending = True, inplace = True)
rv_dataframe = rv_dataframe[0:50]
rv_dataframe.reset_index(drop = True, inplace = True)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,KSS,28.42,N/A,4.34,0.883002,0.8312,1.986755,0.1957,0.441501,3.147396,1.545254,0.956361,4.415011,1.854305
1,AIG,54.57,N/A,3.50,0.441501,0.9390,3.752759,0.7146,10.154525,2.006995,0.441501,0.707717,1.545254,3.267108
2,PVH,63.54,N/A,4.66,1.545254,0.8151,1.545254,0.4684,4.415011,4.744126,7.94702,1.073545,5.298013,4.15011
3,UNM,38.35,N/A,7.14,7.505519,0.8180,1.766004,0.6707,8.830022,3.959529,4.635762,0.664158,1.103753,4.768212
4,GM,39.21,N/A,7.57,8.609272,0.9341,3.311258,0.4442,4.19426,2.408380,0.662252,1.839963,10.154525,5.386313
5,SYF,34.16,N/A,5.00,2.869757,1.2900,9.050773,0.9380,13.686534,2.934661,0.883002,0.972061,4.635762,6.225166
6,COF,108.16,N/A,4.69,1.986755,0.7973,1.103753,1.2100,20.198675,3.258456,2.207506,1.318999,7.064018,6.512141
7,C,51.01,N/A,6.30,4.856512,0.5438,0.662252,1.1600,18.763797,3.728061,3.532009,1.132963,6.401766,6.843267
8,PHM,42.6,N/A,4.62,1.103753,1.2700,8.609272,0.6528,7.505519,3.767864,3.752759,2.644939,16.335541,7.461369
9,LNC,48.24,N/A,9.67,12.913907,0.9058,2.869757,0.4160,2.869757,7.143939,18.763797,0.418081,0.220751,7.527594


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [26]:
portfolio_input()

In [27]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
position_size

500000.0

In [28]:
for row in rv_dataframe.index:
    rv_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe.loc[row, 'Price'])

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,KSS,28.42,17593,4.34,0.883002,0.8312,1.986755,0.1957,0.441501,3.147396,1.545254,0.956361,4.415011,1.854305
1,AIG,54.57,9162,3.50,0.441501,0.9390,3.752759,0.7146,10.154525,2.006995,0.441501,0.707717,1.545254,3.267108
2,PVH,63.54,7869,4.66,1.545254,0.8151,1.545254,0.4684,4.415011,4.744126,7.94702,1.073545,5.298013,4.15011
3,UNM,38.35,13037,7.14,7.505519,0.8180,1.766004,0.6707,8.830022,3.959529,4.635762,0.664158,1.103753,4.768212
4,GM,39.21,12751,7.57,8.609272,0.9341,3.311258,0.4442,4.19426,2.408380,0.662252,1.839963,10.154525,5.386313
5,SYF,34.16,14637,5.00,2.869757,1.2900,9.050773,0.9380,13.686534,2.934661,0.883002,0.972061,4.635762,6.225166
6,COF,108.16,4622,4.69,1.986755,0.7973,1.103753,1.2100,20.198675,3.258456,2.207506,1.318999,7.064018,6.512141
7,C,51.01,9801,6.30,4.856512,0.5438,0.662252,1.1600,18.763797,3.728061,3.532009,1.132963,6.401766,6.843267
8,PHM,42.6,11737,4.62,1.103753,1.2700,8.609272,0.6528,7.505519,3.767864,3.752759,2.644939,16.335541,7.461369
9,LNC,48.24,10364,9.67,12.913907,0.9058,2.869757,0.4160,2.869757,7.143939,18.763797,0.418081,0.220751,7.527594


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [29]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine = 'xlsxwriter')
rv_dataframe.to_excel(writer, 'Value Strategy', index = False )


## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [30]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )


percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [31]:
column_formats = {    
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', float_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile', float_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', float_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', float_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', float_template],
                    'N': ['RV Score', float_template]
}


for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])


## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [32]:
writer.save()